# Nutrient inputs

In this notebook we define the inputs of mineral fertilizer, struvite, compost and ammonium salts before balance per polygon, 
accounting for limitations of production at the AMB and the characteristics of each product. The main data input is the nutrients (NPK) crop requirement from the URBAG map. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
%store -r URBAG_map
%store -r Total_N_require
%store -r Total_P2O5_require
%store -r Total_K2O_require

In [ ]:
len_URBAG_map= len(URBAG_map['Voronoi_1'])
print(f"This is the amount of plots in the map in this notebook: {len_URBAG_map}")

In [ ]:
URBAG_map.keys()

## Inputs assumed to be Zero

In [ ]:
#Filling parameters in the URBAG map for manure and residues, all are set to zero i.e. these inputs are not included.
URBAG_map['Manure']=0
URBAG_map['Manure_N']=0
URBAG_map['Manure_P2O5']=0
URBAG_map['Agriwaste_toField']=0
URBAG_map['Agriwaste_burned']=0
URBAG_map['N_agriwaste_to_field']=0

# Determining N, P, K inputs from different sources

## Struvite

### Content of N and P in struvite

According to Want et.al. (2023)

    *kg P2O5 per kg MAP = 0.25
    *kg N per kg MAP = 0.045

ICTA struvite and ENRICH struvite too:

    *kg P2O5 per kg MAP = 0.29
    *kg N per kg MAP = 0.057

In [ ]:
kg_P_kg_MAP = 0.127
kg_P2O5_kg_MAP = 0.29
kg_MAP_kg_P2O5 = 1/0.29
kg_N_kg_MAP = 0.057
kg_MAP_kg_N = 1/0.057

In [ ]:
tons_struvite_year = 2167

In [ ]:
N_in_struvite = tons_struvite_year*kg_N_kg_MAP
N_in_struvite

In [ ]:
P_in_struvite = tons_struvite_year*kg_P_kg_MAP
P_in_struvite

## Compost

### Content of N, P, K in compost produced in the AMB

See file under data/compost_inventory/Compost_Scenario_Data_JA

N in compost = DM% * N content * Assimilation rate

P in compost = DM% * P content * Assimilation rate

K in compost = DM% * Kcontent * Assimilation rate

Three compost production sites in AMB in 2016: 

    *Sant Cugat CP
    *Torrelles CP
    *Barcelina Zona Franca ECP

In [ ]:
kg_N_kg_FMcompost_SantCugatCP = 0.789*0.025*0.4
kg_P2O5_kg_FMcompost_SantCugatCP = 0.789*0.0057*0.95*(1/0.4364)
kg_K2O_kg_FMcompost_SantCugatCP = 0.789*0.0129*0.95*1.21

In [ ]:
kg_N_kg_FMcompost_TorrellesCP = 0.779*0.0284*0.4
kg_P2O5_kg_FMcompost_TorrellesCP = 0.779*0.0064*0.95*(1/0.4364)
kg_K2O_kg_FMcompost_TorrellesCP = 0.779*0.0195*0.95*1.21

In [ ]:
kg_N_kg_FMcompost_ZonaFrancaECP = 0.592*0.023*0.4
kg_P2O5_kg_FMcompost_ZonaFrancaECP = 0.592*0.0197*0.95*(1/0.4364)
kg_K2O_kg_FMcompost_ZonaFrancaECP = 0.592*0.0091*0.95*1.21

In [ ]:
kg_N_kg_DMcompost_SantCugatCP_2 = 0.025*0.4
kg_N_kg_DMcompost_TorrellesCP_2 = 0.0284*0.4
kg_N_kg_DMcompost_ZonaFrancaECP_2 = 0.023*0.4

In [ ]:
kg_K2O_kg_DMcompost_SantCugatCP_2 = 0.0129*0.95*1.21
kg_K2O_kg_DMcompost_TorrellesCP_2 = 0.0195*0.95*1.21
kg_K2O_kg_DMcompost_ZonaFrancaECP_2 = 0.0091*0.95*1.21

In [ ]:
kg_compostDM_kg_N = 1/np.average([(kg_N_kg_DMcompost_SantCugatCP_2),(kg_N_kg_DMcompost_TorrellesCP_2),(kg_N_kg_DMcompost_ZonaFrancaECP_2)], weights=[0.21,0.15,0.64])
kg_compostDM_kg_N

In [ ]:
kg_compostDM_kg_K = 1/np.average([(kg_K2O_kg_DMcompost_SantCugatCP_2),(kg_K2O_kg_DMcompost_TorrellesCP_2),(kg_K2O_kg_DMcompost_ZonaFrancaECP_2)], weights=[0.21,0.15,0.64])
kg_compostDM_kg_K

### Assumption of supply of compost nutrients per poligon

As compost is a constrained resource and not all demand of N,P,K by crops in the AMB can be covered by locally produced compost: 

We assume that the % of nutrients supplied by compost for each poligon is equal to
the total AMB compost production of N,P,K in the AMB over the total demand of N,P,K. 

In this way only a fraction of the demand is covered and the rest is assumed to be covered by mineral fertilizer.

In [ ]:
#Tons of compost producted per plant in 2016 * Fractions of available nutrients to plants
Total_NCompost_production_SantCugat = 1075.67*kg_N_kg_FMcompost_SantCugatCP
Total_NCompost_production_Torrelles = 728.42*kg_N_kg_FMcompost_TorrellesCP
Total_NCompost_production_ZonaFranca = 3212.56*kg_N_kg_FMcompost_ZonaFrancaECP
Total_NCompost_production = Total_NCompost_production_SantCugat + Total_NCompost_production_Torrelles + Total_NCompost_production_ZonaFranca
Total_NCompost_production #tons of N in compost per year

In [ ]:
Total_N_require

In [ ]:
#Tons N produced over tons N required by crops
FracN_supplied_polygon = Total_NCompost_production/Total_N_require
FracN_supplied_polygon

In [ ]:
#Tons of compost producted per plant in 2016 * Fractions of available nutrients to plants
Total_P2O5Compost_production_SantCugat = 1075.67*kg_P2O5_kg_FMcompost_SantCugatCP
Total_P2O5Compost_production_Torrelles = 728.42*kg_P2O5_kg_FMcompost_TorrellesCP
Total_P2O5Compost_production_ZonaFranca = 3212.56*kg_P2O5_kg_FMcompost_ZonaFrancaECP
Total_P2O5Compost_production = Total_P2O5Compost_production_SantCugat + Total_P2O5Compost_production_Torrelles + Total_P2O5Compost_production_ZonaFranca
Total_P2O5Compost_production

In [ ]:
#Check cell (should be similar to input data in P i.e. 45.94 FInal compost P available (t))
Total_P2O5Compost_production*0.4364

In [ ]:
Total_P2O5_require

In [ ]:
#Tons P2O5 produced over tons P2O5 required by crops
FracP2O5_supplied_polygon = Total_P2O5Compost_production/Total_P2O5_require
FracP2O5_supplied_polygon

In [ ]:
#Tons of compost producted per plant in 2016 * Fractions of available nutrients to plants
Total_K2OCompost_production_SantCugat = 1075.67*kg_K2O_kg_FMcompost_SantCugatCP
Total_K2OCompost_production_Torrelles = 728.42*kg_K2O_kg_FMcompost_TorrellesCP
Total_K2OCompost_production_ZonaFranca = 3212.56*kg_K2O_kg_FMcompost_ZonaFrancaECP
Total_K2OCompost_production = Total_K2OCompost_production_SantCugat + Total_K2OCompost_production_Torrelles + Total_K2OCompost_production_ZonaFranca
Total_K2OCompost_production

In [ ]:
#Check cell (should be similar to input data in K i.e. 37.35 FInal compost K available (t))
Total_K2OCompost_production*(1/1.21)

In [ ]:
Total_K2O_require

In [ ]:
#Tons K2O produced over tons K2O required by crops
FracK2O_supplied_polygon = Total_K2OCompost_production/Total_K2O_require
FracK2O_supplied_polygon

## Recovered ammonium salts

### Assumption of supply of nitrogen from ammonium salts from WWTP per poligon

Similar to compost ammonium salts supply of N is constrained. Therefore we assume: 

We assume that the % of nitrogen supplied by ammonium salts for each poligon is equal to
the total Llobregat WWTP N in ammonium salts production over the total demand of N. 

In this way only a fraction of the demand is covered and the rest is assumed to be covered by mineral fertilizer.

The productions used are:

Struvite (tons/yr) > N content 5.7% and P content 12.7% / P2O5 content 29%

    - Fertilizer = 2167
    - P = 274
    - N = 123.83

Ammonium salts (tons/yr) > N content 18.8%

    - Fertilizer = 3475
    - P = 0
    - N = 654

In [ ]:
tons_ammonium_sulphate_year = 2816

In [ ]:
N_content_ammonium_sulphate = 0.188

In [ ]:
N_in_ammonium_salts_year = (tons_ammonium_sulphate_year*N_content_ammonium_sulphate)-N_in_struvite
N_in_ammonium_salts_year

In [ ]:
FracN_supplied_polygon_ammon_salts = N_in_ammonium_salts_year/Total_N_require
FracN_supplied_polygon_ammon_salts

## Inputs of: Mineral Fertilizers, struvite and compost per scenario

### Phosphorus 

In [ ]:
#Defining key variable for phosphorus inputs
if scen == 'S0_MinFert':    
    Mineral_fert_P2O5      = URBAG_map['kgP2O5/ha'] #Total P demand from mineral fertilizers application (kg P2O5/ha)
    Struvite_P2O5          = np.repeat(0, len_URBAG_map) #No struvite in Minfert scenario
    Compost_P2O5           = np.repeat(0, len_URBAG_map) #No compost in Minfert scenario
elif scen == 'S0_struvite_P':
    Mineral_fert_P2O5      = pd.Series(np.repeat(0, len_URBAG_map)) #No P mineral fertilizer in struvite P scenario
    Struvite_P2O5          = URBAG_map['kgP2O5/ha'] #Total P demand covered with struvite (kg P2O5/ha)
    Compost_P2O5           = pd.Series(np.repeat(0, len_URBAG_map)) #No P from compost in struvite scenario
elif scen == 'S0_compost':
    Mineral_fert_P2O5      = URBAG_map['kgP2O5/ha'] * (1-FracP2O5_supplied_polygon) #Difference not supplied from compost supplied from mineral fert (kg P2O5/ha)
    Struvite_P2O5          = pd.Series(np.repeat(0, len_URBAG_map)) #No P from struvite in compost scenario
    Compost_P2O5           = URBAG_map['kgP2O5/ha'] * FracP2O5_supplied_polygon #Total P demand supplied from compost, same fraction assumed for all polygons (kg P2O5/ha)
elif scen == 'S0_Ammon_salts':
    Mineral_fert_P2O5      = pd.Series(np.repeat(0, len_URBAG_map)) #No P from mineral fertilizer in ammonium salts scenario
    Struvite_P2O5          = URBAG_map['kgP2O5/ha'] #Total P demand covered with struvite (kg P2O5/ha)
    Compost_P2O5           = pd.Series(np.repeat(0, len_URBAG_map)) #No P from mineral fertilizer in ammonium salts scenario
else:
    pass

In [ ]:
#Min, Max, average kg of nutrient per ha in selected scenario and lenght of data should be equal to total plots
d = {'P2O5_min': [np.min(Mineral_fert_P2O5), np.min(Struvite_P2O5), np.min(Compost_P2O5), 0], 
     'P2O5_max': [np.max(Mineral_fert_P2O5), np.max(Struvite_P2O5), np.max(Compost_P2O5), 0],
     'P2O5_average': [np.average(Mineral_fert_P2O5), np.average(Struvite_P2O5), np.average(Compost_P2O5), 0],
     'plots': [len(Mineral_fert_P2O5), len(Struvite_P2O5), len(Compost_P2O5), len(Struvite_P2O5)],
    }
df = pd.DataFrame(data=d, index=["MinFert", "Struvite","Compost","Ammon_salts"])
df

In [ ]:
#Inputs for mineral fert, struvite and compost for each plot
d1 = {'P2O5_MinFert': Mineral_fert_P2O5, 
     'P2O5_Struvite': Struvite_P2O5,
     'P2O5_Compost': Compost_P2O5,
     'P2O5_Ammon_salts': 0,
    }
df1 = pd.DataFrame(data=d1)
df1

In [ ]:
sns.set()

In [ ]:
sns.boxplot(data=df1)
plt.title('Inputs of P - kg P2O5 per ha yr')

### Nitrogen

In [ ]:
#Defining key variables for nitrogen inputs
if scen == 'S0_MinFert':    
    Mineral_fert_N      = URBAG_map['kgN/ha'] #Total N demand from mineral fertilizers application (kg N/ha)
    Struvite_N          = np.repeat(0, len_URBAG_map) #No N from struvite in Minfert scenario
    Compost_N           = np.repeat(0, len_URBAG_map) #No N from compost in Minfert scenario
    Ammonium_salts      = np.repeat(0, len_URBAG_map) #No N from compost in Minfert scenario
elif scen == 'S0_struvite_P':
    Mineral_fert_N      = URBAG_map['kgN/ha'] - (URBAG_map['kgP2O5/ha']*kg_MAP_kg_P2O5*kg_N_kg_MAP) #Difference of total N demand minus N covered by struvite to cover P (kg N/ha)
    Struvite_N          = URBAG_map['kgP2O5/ha']*kg_MAP_kg_P2O5*kg_N_kg_MAP #N demand covered by supply of struvite to cover P given N content of struvite (kg N/ha)
    Compost_N           = pd.Series(np.repeat(0, len_URBAG_map)) #No N from compost in struvite scenario
    Ammonium_salts      = np.repeat(0, len_URBAG_map) #No N from compost in Minfert scenario
elif scen == 'S0_compost':
    Mineral_fert_N      = URBAG_map['kgN/ha']* (1-FracN_supplied_polygon) #Total N demand not supplied from compost supplied by MinFert (kg N/ha)
    Struvite_N          = pd.Series(np.repeat(0, len_URBAG_map)) #No N from struvite in compost scenario
    Compost_N           = URBAG_map['kgN/ha']*FracN_supplied_polygon #Total N demand supplied from compost, same fraction assumed for all polygons (kg N/ha)
    Ammonium_salts      = np.repeat(0, len_URBAG_map) #No N from compost in Minfert scenario
if scen == 'S0_Ammon_salts':    
    Mineral_fert_N      = (URBAG_map['kgN/ha'] - (URBAG_map['kgP2O5/ha']*kg_MAP_kg_P2O5*kg_N_kg_MAP))* (1-FracN_supplied_polygon_ammon_salts) #Total N demand minus what comes from struvite times fraction not covered from ammonium salts from mineral fertilizers application (kg N/ha)
    Struvite_N          = URBAG_map['kgP2O5/ha']*kg_MAP_kg_P2O5*kg_N_kg_MAP #N demand covered by supply of struvite to cover P given N content of struvite (kg N/ha)
    Compost_N           = np.repeat(0, len_URBAG_map) #No N from compost in Minfert scenario
    Ammonium_salts      = (URBAG_map['kgN/ha'] - (URBAG_map['kgP2O5/ha']*kg_MAP_kg_P2O5*kg_N_kg_MAP))* FracN_supplied_polygon_ammon_salts #Total N demand minus what comes from struvite times fraction covered from ammonium salts (kg N/ha)
else:
    pass

In [ ]:
#Min, Max, average kg of nutrient per ha in selected scenario and lenght of data should be equal to total plots
d2 = {'N_min': [np.min(Mineral_fert_N), np.min(Struvite_N), np.min(Compost_N), np.min(Ammonium_salts)], 
     'N_max': [np.max(Mineral_fert_N), np.max(Struvite_N), np.max(Compost_N), np.max(Ammonium_salts)],
     'N_average': [np.average(Mineral_fert_N), np.average(Struvite_N), np.average(Compost_N), np.average(Ammonium_salts)],
     'plots': [len(Mineral_fert_N), len(Struvite_N), len(Compost_N), len(Ammonium_salts)],
    }
df2 = pd.DataFrame(data=d2, index=["MinFert", "Struvite","Compost","Ammon_salts"])
df2

In [ ]:
d3 = {'N_MinFert': Mineral_fert_N, 
     'N_Struvite': Struvite_N,
     'N_Compost': Compost_N,
      'N_Ammon_salts': Ammonium_salts,
    }
df3 = pd.DataFrame(data=d3)
df3

In [ ]:
sns.boxplot(data=df3, showfliers=False)
plt.title('Inputs of N - kg N per ha yr')

### Potassium

In [ ]:
#Defining key variables for potassium inputs
if scen == 'S0_MinFert':    
    Mineral_fert_K2O      = URBAG_map['kgK2O/ha'] #Total K demand from mineral fertilizers application (kg K2O/ha)
    Struvite_K2O          = np.repeat(0, len_URBAG_map) #No K from struvite in Minfert scenario
    Compost_K2O           = np.repeat(0, len_URBAG_map) #No K from compost in Minfert scenario
elif scen == 'S0_struvite_P':
    Mineral_fert_K2O      = URBAG_map['kgK2O/ha'] #otal K demand from mineral fertilizers application in struvite scenario as no K in struvite (kg K2O/ha)
    Struvite_K2O          = pd.Series(np.repeat(0, len_URBAG_map)) #No K from struvite in struvite scenario
    Compost_K2O           = pd.Series(np.repeat(0, len_URBAG_map)) #No K from compost in struvite scenario
elif scen == 'S0_compost':
    Mineral_fert_K2O      = URBAG_map['kgK2O/ha']*(1-FracK2O_supplied_polygon) #Total K demand not supplied from compost supplied by MinFert (kg K2O/ha)
    Struvite_K2O          = pd.Series(np.repeat(0, len_URBAG_map)) #No K from struvite in compost scenario
    Compost_K2O           = URBAG_map['kgK2O/ha']*FracK2O_supplied_polygon #Total K demand supplied from compost, same fraction assumed for all polygons (kg K2O/ha)
if scen == 'S0_Ammon_salts':    
    Mineral_fert_K2O      = URBAG_map['kgK2O/ha'] #Total K demand from mineral fertilizers application (kg K2O/ha)
    Struvite_K2O          = np.repeat(0, len_URBAG_map) #No K from struvite in Minfert scenario
    Compost_K2O           = np.repeat(0, len_URBAG_map) #No K from compost in Minfert scenario
else:
    pass

In [ ]:
#Min, Max, average kg of nutrient per ha in selected scenario and lenght of data should be equal to total plots
d4 = {'K_min': [np.min(Mineral_fert_K2O), np.min(Struvite_K2O), np.min(Compost_K2O), 0], 
     'K_max' : [np.max(Mineral_fert_K2O), np.max(Struvite_K2O), np.max(Compost_K2O), 0],
     'K_average': [np.average(Mineral_fert_K2O), np.average(Struvite_K2O), np.average(Compost_K2O), 0],
     'plots': [len(Mineral_fert_K2O), len(Struvite_K2O), len(Compost_K2O), len(Struvite_K2O)],
    }
df4 = pd.DataFrame(data=d4, index=["MinFert", "Struvite","Compost","Ammon_salts"])
df4

In [ ]:
d5 = {'K2O_MinFert': Mineral_fert_K2O, 
     'K2O_Struvite': Struvite_K2O,
     'K2O_Compost': Compost_K2O,
      'K2O_Ammon_salts': 0,
    }
df5 = pd.DataFrame(data=d5)
df5

In [ ]:
sns.boxplot(data=df5, showfliers=True)
plt.title('Inputs of K - kg K2O per ha yr')

### Variables for Manure and residues

In [ ]:
Manure_N = URBAG_map['Manure_N']
Manure_N

In [ ]:
N_agriwaste_to_field = URBAG_map['N_agriwaste_to_field']
N_agriwaste_to_field

In [ ]:
Manure_P2O5 = URBAG_map['Manure_P2O5']
Manure_P2O5

In [ ]:
URBAG_map.keys()

In [ ]:
URBAG_map.loc[URBAG_map['Fid']==547]

In [ ]:
Mineral_fert_P2O5.loc[Mineral_fert_P2O5.index==547]

## Saving outputs

In [ ]:
#Variables for next notebook
%store URBAG_map

%store Mineral_fert_N
%store Mineral_fert_P2O5
%store Mineral_fert_K2O
%store Ammonium_salts

%store Struvite_N
%store Struvite_P2O5
%store Struvite_K2O

%store Compost_N
%store Compost_P2O5
%store Compost_K2O

%store Manure_N
%store Manure_P2O5

%store N_agriwaste_to_field